## 데이터

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import sqlite3
import re

In [3]:
df_2024 = pd.read_csv('/content/drive/MyDrive/1_public/markets_data/markets_2024.csv', encoding='euc-kr') # utf-8

In [4]:
df_2024.columns # 2024년 데이터

Index(['시장코드', '시장명', '시장 유형', '지번주소', '도로명주소', '시도', '시군구', '아케이드 보유 여부',
       '엘리베이터_에스컬레이터_보유여부', '고객지원센터 보유 여부', '스프링쿨러 보유 여부', '화재감지기 보유여부',
       '유아놀이방_보유여부', '종합콜센터_보유여부', '고객휴게실_보유여부', '수유센터_보유여부', '물품보관함_보유여부',
       '자전거보관함_보유여부', '체육시설_보유여부', '간이 도서관_보유여부', '쇼핑카트_보유여부', '외국인 안내센터_보유여부',
       '고객동선통로_보유여부', '방송센터_보유여부', '문화교실_보유여부', '공동물류창고_보유여부',
       '시장전용 고객주차장_보유여부', '교육장_보유여부', '회의실_보유여부', '자동심장충격기_보유여부'],
      dtype='object')

In [5]:
df_2024.head()

,시장코드,시장명,시장 유형,지번주소,도로명주소,시도,시군구,아케이드 보유 여부,엘리베이터_에스컬레이터_보유여부,고객지원센터 보유 여부,...,쇼핑카트_보유여부,외국인 안내센터_보유여부,고객동선통로_보유여부,방송센터_보유여부,문화교실_보유여부,공동물류창고_보유여부,시장전용 고객주차장_보유여부,교육장_보유여부,회의실_보유여부,자동심장충격기_보유여부
0,b31370001,설악눈메골시장,전통시장,경기도 가평군 설악면 신천리 413-14,경기도 가평군 설악면 신천중앙로 104-1,경기도,가평군,N,N,N,...,N,N,N,N,N,N,Y,Y,Y,N
1,a31370001,가평잣고을시장,전통시장,경기도 가평군 읍내리 405,경기도 가평군 가평읍 장터2길12 204호,경기도,가평군,Y,N,Y,...,N,N,Y,Y,N,Y,Y,Y,Y,N
2,a31370007,청평여울시장,전통시장,경기도 가평군 청평면 청평리 81-2,경기도 가평군 청평면 시장중앙로19,경기도,가평군,Y,Y,N,...,N,N,N,N,N,N,Y,N,Y,N
3,a31100001,원당시장,전통시장,경기도 고양시 덕양구 성사동 697-14,경기도 고양시 덕양구 호국로 790번길 17,경기도,고양시,Y,N,Y,...,N,N,Y,Y,N,N,Y,Y,Y,Y
4,a31101001,능곡시장,전통시장,경기도 고양시 덕양구 토당동 347,경기도 고양시 덕양구 능곡로 12의 9,경기도,고양시,N,N,Y,...,Y,N,Y,Y,N,Y,Y,Y,Y,Y


In [6]:
df_2024.columns

Index(['시장코드', '시장명', '시장 유형', '지번주소', '도로명주소', '시도', '시군구', '아케이드 보유 여부',
       '엘리베이터_에스컬레이터_보유여부', '고객지원센터 보유 여부', '스프링쿨러 보유 여부', '화재감지기 보유여부',
       '유아놀이방_보유여부', '종합콜센터_보유여부', '고객휴게실_보유여부', '수유센터_보유여부', '물품보관함_보유여부',
       '자전거보관함_보유여부', '체육시설_보유여부', '간이 도서관_보유여부', '쇼핑카트_보유여부', '외국인 안내센터_보유여부',
       '고객동선통로_보유여부', '방송센터_보유여부', '문화교실_보유여부', '공동물류창고_보유여부',
       '시장전용 고객주차장_보유여부', '교육장_보유여부', '회의실_보유여부', '자동심장충격기_보유여부'],
      dtype='object')

In [7]:
df_st_2024 = pd.read_csv('/content/drive/MyDrive/1_public/markets_data/markets_st_2024.csv', encoding='euc-kr') # utf-8

In [8]:
df_st_2024.columns # 2015년 데이터

Index(['시장명', '시장유형', '소재지도로명주소', '시장개설주기', '소재지지번주소', '점포수', '사용가능상품권',
       '공중화장실 보유여부', '주차장 보유여부', '개설년도', '데이터기준일자'],
      dtype='object')

In [9]:
df_st_2024['사용가능상품권'].unique()

array(['온누리상품권', nan], dtype=object)

In [10]:
df_st_2024.head()

,시장명,시장유형,소재지도로명주소,시장개설주기,소재지지번주소,점포수,사용가능상품권,공중화장실 보유여부,주차장 보유여부,개설년도,데이터기준일자
0,중앙시장,상설,강원도 강릉시 금성로 21,상설,강원 강릉시 성남동 50번지,229,온누리상품권,Y,Y,1980.0,2015-12-31
1,강릉동부시장,상설,강원도 강릉시 옥천로 48,상설,강원 강릉시 옥천동 223번지,91,NaN,Y,N,2014.0,2015-12-31
2,강릉서부시장,상설,강원도 강릉시 임영로 155번길 6,상설,강원 강릉시 용강동 29번지,109,온누리상품권,Y,Y,1977.0,2015-12-31
3,주문진종합시장,상설+정기,강원도 강릉시 주문진읍 구 시장길,5일,강원 강릉시 주문진읍 주문리 321-52번지,77,온누리상품권,Y,Y,2006.0,2015-12-31
4,주문진수산시장,상설,강원도 강릉시 주문진읍 시장1길 4-1,상설,강원 강릉시 주문진읍 주문리 312-91번지,42,온누리상품권,Y,Y,2007.0,2015-12-31


---

## 시장명 서로 일치하는지 확인

In [11]:
def clean_market_name(name):
    # 1. 괄호 안 내용 제거 (예: "상남시장(상설+5일장)" → "상남시장")
    name = re.sub(r'\([^)]*\)', '', name)
    # 2. 느낌표나 특수기호 제거 (예: "시장!" → "시장")
    name = re.sub(r'[^\w가-힣]', '', name)  # 한글/영문/숫자 외 제거
    # 3. 공백 및 하이픈 제거 (이미 2에서 대부분 처리되지만 추가로 안전 조치)
    name = name.strip().replace(' ', '').replace('-', '')
    # # 4. 의미 없는 일반 단어 제거
    # stopwords = ['시장', '전통', '종합', '공설', '중앙', '상가', '민속', '현대', '시장상가']
    # for sw in stopwords:
    #     name = name.replace(sw, '')

    return name

# 우선 각 DataFrame에서 '시장명' 열 추출
market_names_st_2024 = set(df_st_2024['시장명'].map(clean_market_name))
market_names_2024 = set(df_2024['시장명'].map(clean_market_name))

# 1. 각 쌍끼리 공통된 시장명
common_st_and_2024 = market_names_st_2024 & market_names_2024

# 2. 세 집합 전체 공통 시장명
common_all = market_names_st_2024 & market_names_2024

# 3. 집합 간 차이 확인 (어떤 시장이 누락됐는지 등)
only_in_st_2024 = market_names_st_2024 - market_names_2024
only_in_2024 = market_names_2024 - market_names_st_2024

# 결과 출력 예시
print(f"두 파일 모두에 공통으로 존재하는 시장 수: {len(common_all)}")
print(f"st_2024에만 존재하는 시장 수: {len(only_in_st_2024)}")
print(f"2024에만 존재하는 시장 수: {len(only_in_2024)}")

두 파일 모두에 공통으로 존재하는 시장 수: 1065
st_2024에만 존재하는 시장 수: 292
2024에만 존재하는 시장 수: 269


In [12]:
print("st_2024에만 있는 시장:", only_in_st_2024)
print("2024에만 있는 시장:", only_in_2024)

st_2024에만 있는 시장: {'대연삼익상가시장', '동해종합시장', '태안특산물전통시장', '효목시장', '건어물시장', '고산시장', '남목시티시장', '노화읍5일시장', '춘천남부시장', '서중시장', '고분다리골목시장', '수영팔도상가시장', '진상5일시장', '인천산업유통사업협동조합', '사당시장', '가평잣고을전통시장', '세흥시장', '호계종합시장', '사북시장', '장위시장', '르네시떼', '벽진시장', '보성황실시장', '동아종합시장', '예당5일시장', '영해시장', '천호시장', '칠성로상가', '도림시장', '산성전통시장', '석관황금시장', '나산시장', '대림골목시장', '김포중앙시장', '전의전통시장', '인천용현시장', '신광시장', '현대타운상가', '회천5일시장', '서울남대문시장', '펭귄시장', '대성시장', '영진시장', '송강시장', '망미종합시장', '아랫장', '일반성상가', '탑동유니코상가', '화순전통시장', '성원3차상가', '부산진남문시장', '대산시장', '광명시장', '송도시장', '문화시장', '고강시장', '완도중앙시장', '선어시장', '토월복합상가', '야탑에이스코아시장', '숭의평화시장', '남문시장', '강북종합전통시장', '울진시장', '선산종합시장', '전주동부시장', '임계시장', '반송2동시장', '상하시장', '진천전통시장', '망운전통시장', '세화민속오일시장', '봉곡시장', '거성시장', '칠성원시장', '골목시장', '유신종합상가', '임회면십일시장', '장성종합시장', '복현종합시장', '송정역전매일시장', '흥덕시장', '동서울시장', '사러가시장', '거진시장', '논현종합시장', '당진시장', '진해중부시장', '복대시장', '내토재래시장', '무장시장', '성원그랜드쇼핑', '영춘전통시장', '원종시장', '서방시장', '죽전로데오상점가전통시장', '연무안심시장', '마천전통시장', '간성시장', '영등포기계상가', '용문전통시장', '성동시장', '신성시장', '금남

## 각 데이터 프레임에서 정제해서 시장 이름 통일
> - 시장명과 주소 정규화
> - 문자 유사도 기반으로 매치

In [13]:
import pandas as pd
from difflib import get_close_matches, SequenceMatcher

import pandas as pd
from difflib import SequenceMatcher

# 주소 정규화 함수
def normalize_address(addr):
    if pd.isna(addr):
        return ""
    addr = addr.replace("서울시", "서울특별시")
    addr = addr.replace(" ", "")
    return addr

# 시장명 정규화 함수
def normalize_market(name):
    # 1. 괄호 안 내용 제거 (예: "상남시장(상설+5일장)" → "상남시장")
    name = re.sub(r'\([^)]*\)', '', name)
    # 2. 느낌표나 특수기호 제거 (예: "시장!" → "시장")
    name = re.sub(r'[^\w가-힣]', '', name)  # 한글/영문/숫자 외 제거
    # 3. 공백 및 하이픈 제거 (이미 2에서 대부분 처리되지만 추가로 안전 조치)
    name = name.strip().replace(' ', '').replace('-', '')
    # 4. 의미 없는 일반 단어 제거
    stopwords = ['전통']
    for sw in stopwords:
        name = name.replace(sw, '')
    return name

# 문자열 유사도 계산 함수
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# 주소 + 시장명 모두 유사한 경우만 변경
def update_market_names_with_double_similarity(df_st, df_base, addr_cutoff=0.85, name_cutoff=0.55):
    df_st = df_st.copy()
    df_base = df_base.copy()

    # 주소 정규화
    df_st['소재지도로명주소_정규화'] = df_st['소재지도로명주소'].apply(normalize_address)
    df_base['도로명주소_정규화'] = df_base['도로명주소'].apply(normalize_address)

    df_st['시장명_정규화'] = df_st['시장명'].apply(normalize_market)
    df_base['시장명_정규화'] = df_base['시장명'].apply(normalize_market)


    # 비교용 리스트 생성
    updated_names = []
    original_names = df_st['시장명'].tolist()
    base_market_info = df_base[['시장명', '시장명_정규화', '도로명주소_정규화']].values.tolist()

    for idx, row in df_st.iterrows():
        st_name_raw = row['시장명']
        st_name_norm = row['시장명_정규화']
        st_addr_norm = row['소재지도로명주소_정규화']

        best_match = None
        best_score = 0

        for base_name, base_name_norm, base_addr_norm in base_market_info:
            addr_sim = similar(st_addr_norm, base_addr_norm)
            name_sim = similar(st_name_norm, base_name_norm)

            if addr_sim >= addr_cutoff and name_sim >= name_cutoff:
                score = (addr_sim + name_sim) / 2
                if score > best_score:
                    best_score = score
                    best_match = base_name

        updated_names.append(best_match if best_match else st_name_raw)

    df_st['시장명_원본'] = original_names
    df_st['시장명'] = updated_names
    df_st['시장명_변경됨'] = df_st['시장명_원본'] != df_st['시장명']

    return df_st

# ✅ 사용 예시
# df_2024: 최신 기준 (시장명, 도로명주소)
# df_st_2024: 수정 대상 (시장명, 소재지도로명주소)
df_st_updated = update_market_names_with_double_similarity(df_st_2024, df_2024)

# ✅ 결과 확인
changed = df_st_updated['시장명_변경됨'].sum()
nan_count = df_st_updated['시장명'].isnull().sum()

print(f"시장명 변경된 수: {changed}")
print(f"NaN (매칭 실패) 수: {nan_count}")

# 변경된 예시 출력
print(df_st_updated[df_st_updated['시장명_변경됨']][['시장명_원본', '시장명']])

시장명 변경된 수: 86
NaN (매칭 실패) 수: 0
                  시장명_원본                 시장명
8                   거진시장              거진전통시장
21                영월서부시장              서부아침시장
27                  중앙시장  미로예술원주중앙시장(원주중앙시장)
33                  사북시장              정선사북시장
39                  외수시장                와수시장
...                  ...                 ...
1402  제천역전한마음시장(구제천역전시장)           제천역전한마음시장
1410   진천전통시장 (구,진천재래시장)      운수대통! 생거진천전통시장
1421              내덕자연시장   밤고개자연시장(구 내덕자연시장)
1429   농산물종합시장(구,중앙청과시장)   충주농산물종합시장(중앙청과시장)
1435              중앙공설시장             중앙어울림시장

[86 rows x 2 columns]


### ✅ 일부 시장명 교체 목록
| 기존 시장명            | 교체할 시장명                |
| ------------------ | ------------------ |
| 거진시장               | 거진전통시장             |
| 영월서부시장             | 서부아침시장             |
| 중앙시장               | 미로예술원주중앙시장(원주중앙시장) |
| 사북시장               | 정선사북시장             |
| 외수시장               | 와수시장               |
| 제천역전한마음시장(구제천역전시장) | 제천역전한마음시장          |
| 진천전통시장 (구,진천재래시장)  | 운수대통! 생거진천전통시장     |
| 내덕자연시장             | 밤고개자연시장(구 내덕자연시장)  |
| 농산물종합시장(구,중앙청과시장)  | 충주농산물종합시장(중앙청과시장)  |
| 중앙공설시장             | 중앙어울림시장            |


## 시장명 기준으로 병합

In [14]:
df_st_updated.columns

Index(['시장명', '시장유형', '소재지도로명주소', '시장개설주기', '소재지지번주소', '점포수', '사용가능상품권',
       '공중화장실 보유여부', '주차장 보유여부', '개설년도', '데이터기준일자', '소재지도로명주소_정규화', '시장명_정규화',
       '시장명_원본', '시장명_변경됨'],
      dtype='object')

In [15]:
df_2024.columns

Index(['시장코드', '시장명', '시장 유형', '지번주소', '도로명주소', '시도', '시군구', '아케이드 보유 여부',
       '엘리베이터_에스컬레이터_보유여부', '고객지원센터 보유 여부', '스프링쿨러 보유 여부', '화재감지기 보유여부',
       '유아놀이방_보유여부', '종합콜센터_보유여부', '고객휴게실_보유여부', '수유센터_보유여부', '물품보관함_보유여부',
       '자전거보관함_보유여부', '체육시설_보유여부', '간이 도서관_보유여부', '쇼핑카트_보유여부', '외국인 안내센터_보유여부',
       '고객동선통로_보유여부', '방송센터_보유여부', '문화교실_보유여부', '공동물류창고_보유여부',
       '시장전용 고객주차장_보유여부', '교육장_보유여부', '회의실_보유여부', '자동심장충격기_보유여부'],
      dtype='object')

### df_2024와 df_2015 비교

| 컬럼명          | 설명             | df\_2024에 있는지?                  | 추천 사유                      |
| ------------ | -------------- | ------------------------------- | -------------------------- |
| `시장유형`       | 시장의 분류         | ✔ `시장 유형`으로 존재 (컬럼명만 다름)        | ❌ 이미 있음                    |
| `소재지도로명주소`   | 주소             | ✔ `도로명주소`로 존재                   | ❌ 중복                       |
| `소재지지번주소`    | 주소 (지번)        | ✔ `지번주소`로 존재                    | ❌ 중복                       |
| `시장개설주기`     | 정기시장 여부        | ✅                             | ✅ **정기시장/상설시장 여부**는 유의미    |
| `점포수`        | 점포 개수          | ✅ df_2024_st에 있음                            | ✅ **시장 규모 판단 지표**로 유의미     |
| `사용가능상품권`    | 온누리상품권 등 가능 여부 | ✅ df_2024_st에 있음                            | ✅ **결제 수단 다양성 정보**로 유용     |
| `공중화장실 보유여부` | 편의시설 정보        | ✅ df_2024_st에 있음                            | ✅ **화장실 여부는 핵심 편의시설** 중 하나 |
| `주차장 보유여부`   | 주차장 유무         | ✅ df_2024_st에 있음 | ⚠️ **유사**, 비교용으로는 OK       |
| `개설년도`       | 시장 연도          | ❌ 없음                            | ✅ **시장 연령** 분석에 유용         |
| `데이터기준일자`    | 수집 시점          | ❌ 없음                            | ⚠️ 덜 중요, 보조 정보용            |


### 중복된 컬럼 - 시장유형, 주차장보유여부

In [16]:
# 중복 제거된 시장 유형 집합
set_st = set(df_st_updated['시장유형'].dropna().unique())
set_24 = set(df_2024['시장 유형'].dropna().unique())

# 차이 확인
only_in_st = set_st - set_24
only_in_24 = set_24 - set_st

print(f"df_st_2024에만 있는 시장유형: {only_in_st}")
print(f"df_2024에만 있는 시장유형: {only_in_24}") # @ df_st_2024 데이터의 시장유형 선택

df_st_2024에만 있는 시장유형: {'상설+정기', '정기', '상설'}
df_2024에만 있는 시장유형: {'전통시장'}


In [17]:
# 중복 제거된 고객주차장 집합
set_st = set(df_st_updated['주차장 보유여부'].dropna().unique())
set_24 = set(df_2024['시장전용 고객주차장_보유여부'].dropna().unique())

# 차이 확인
only_in_st = set_st - set_24
only_in_24 = set_24 - set_st

print(f"df_st_2024에만 있는 시장명: {only_in_st}")
print(f"df_2024에만 있는 시장명: {only_in_24}") # @ 모두 통일되있음

df_st_2024에만 있는 시장명: set()
df_2024에만 있는 시장명: set()


### df_2024와 df_2024_st 비교


| 컬럼명                 | 출처(csv)      | 설명                | 분석 활용 아이디어                |
| ------------------- | ------------ | ----------------- | ------------------------- |
| 시장명                 | 공통           | 병합 기준             | 시장별 병합 및 분석 단위 기준         |
| 시장유형                | `df_st_2024` | 전통시장, 상설장 등 상세 유형 | 유형별 만족도/시설 비교             |
| 소재지도로명주소            | `df_st_2024` | 시장 도로명 주소         | 리뷰 수집·지도 시각화              |
| 소재지지번주소             | `df_st_2024` | 시장 지번 주소          | 백업 주소용, 위치 보완             |
| 시장개설주기              | `df_st_2024` | 상설 vs 정기시장 구분     | 운영 방식 따른 리뷰/방문수 차이 분석     |
| 위도 / 경도             | `df_st_2024` | 지도 시각화용 좌표        | 지역별 분포 시각화 (folium 등)     |
| 점포수                 | `df_st_2024` | 시장 규모             | 규모 대비 평점 분석, 랭킹 기준        |
| 취급품목                | `df_st_2024` | 식료품, 잡화 등         | 품목별 선호 분석, 클러스터링 등        |
| 사용가능상품권             | `df_st_2024` | 온누리상품권 등          | 결제 편의성에 따른 만족도 비교         |
| 공중화장실보유여부           | `df_st_2024` | 공용 편의시설 유무        | 시설 유무별 리뷰 언급 차이 비교        |
| 주차장보유여부             | `df_st_2024` | 주차장 여부            | 교통 편의성 영향 분석              |
| 전화번호                | `df_st_2024` | 시장 연락처            | 수동 확인/보조 자료용              |
| 아케이드 보유 여부          | `df_2024`    | 아케이드 지붕 유무        | 쾌적성 관련 만족도 분석             |
| 엘리베이터\_에스컬레이터\_보유여부 | `df_2024`    | 접근 편의성            | 고령자 친화 여부                 |
| 고객지원센터 보유 여부        | `df_2024`    | 서비스 편의            | 고객중심 인프라 평가 요소            |
| 스프링쿨러 보유 여부         | `df_2024`    | 화재 안전 시설          | 안전성과 시장 신뢰도 관계            |
| 화재감지기 보유여부          | `df_2024`    | 화재 감지장치 여부        | 위와 동일                     |
| 유아놀이방\_보유여부         | `df_2024`    | 어린이 편의 시설         | 가족 단위 고객 유입과 리뷰 키워드 연관 분석 |
| 종합콜센터\_보유여부         | `df_2024`    | 통합 전화 서비스         | 외국인·노년층 편의 요소             |
| 고객휴게실\_보유여부         | `df_2024`    | 쉼터 유무             | 체류 시간 관련 리뷰 영향 요소         |
| 수유센터\_보유여부          | `df_2024`    | 아기 동반 고객 편의       | 키워드 '아기', '엄마' 분석 관련      |
| 물품보관함\_보유여부         | `df_2024`    | 짐 보관 편의           | 쇼핑 편의성 평가                 |
| 자전거보관함\_보유여부        | `df_2024`    | 교통수단 이용 고객 편의     | 교통 접근성 분석 요소              |
| 체육시설\_보유여부          | `df_2024`    | 부대시설 다양성          | 종합문화공간으로서의 기능 평가          |
| 간이 도서관\_보유여부        | `df_2024`    | 복합문화 시설           | 문화 키워드 분석 기반              |
| 쇼핑카트\_보유여부          | `df_2024`    | 대형쇼핑형 여부          | 시장 현대화 관련                 |
| 외국인 안내센터\_보유여부      | `df_2024`    | 외국인 친화적 인프라       | 관광시장 평가                   |
| 문화교실\_보유여부          | `df_2024`    | 체험형 콘텐츠 제공        | 체험 키워드 연관성 분석             |
| 자동심장충격기\_보유여부       | `df_2024`    | 응급시설 보유           | 고령자 대상 시장 평가 항목           |

### 파생속성 시장ID 만들기
> - 시도 + 시군구 + 시장명

In [18]:
print(f"차원확인 df_204: {df_2024.shape}")
print(f"차원확인 df_st_2024: {df_st_updated.shape}")

차원확인 df_204: (1388, 30)
차원확인 df_st_2024: (1439, 15)


In [19]:
def extract_sido_sigungu(row, road_col='도로명주소', jibun_col='지번주소', fallback_gu=True):
    """
    행(row)에서 도로명주소/지번주소를 사용하여 시도와 시군구를 추출.
    - road_col: 도로명주소 컬럼명
    - jibun_col: 지번주소 컬럼명
    - fallback_gu: 시군구만 있을 때 서울특별시 등 보정할지 여부
    """

    def extract(addr):
        if pd.isnull(addr) or str(addr).strip() in ['0', '없음', '정보 없음']:
            return None, None

        # 공백으로 분리하여 앞 두 토큰 추출
        tokens = str(addr).strip().split()
        if len(tokens) >= 2:
            return tokens[0], tokens[1]
        return None, None

    # 1. 도로명주소에서 추출 시도
    sido, sigungu = extract(row.get(road_col))

    # 2. 도로명주소 실패 시 지번주소에서 추출 시도
    if not sigungu:
        sido2, sigungu2 = extract(row.get(jibun_col))
        sido = sido or sido2
        sigungu = sigungu or sigungu2

    # 3. 시군구만 있고 시도가 없을 때 보정 (옵션)
    if fallback_gu and not sido and sigungu in ['중구', '종로구', '강북구', '성동구', '동작구', '강남구']:
        sido = '서울시'

    return pd.Series([sido, sigungu])

### 시도명 통일 함수

In [20]:
# 시도명 통일 함수
def normalize_sido(sido):
    if sido == '전북특별자치도':
        return '전라북도'
    elif sido == '강원특별자치도':
        return '강원도'
    elif sido == '대구광역시' and '군위군' in sido: # 군위군 편입
        return '경상북도'
    return sido

In [21]:
# @ 1. df_st_updated
df_st_updated[['시도', '시군구']] = df_st_updated.apply(
    lambda row: extract_sido_sigungu(
        row, road_col='소재지도로명주소', jibun_col='소재지지번주소'
    ), axis=1
)

# df_st_updated 시도명 처리
df_st_updated['시도'] = df_st_updated['시도'].apply(normalize_sido)

df_st_updated['시장ID'] = df_st_updated['시도'] + ' ' + df_st_updated['시군구'] + ' ' + df_st_updated['시장명']

In [22]:
# @ 2. df_2024
df_2024[['시도', '시군구']] = df_2024.apply(
    lambda row: extract_sido_sigungu(
        row, road_col='도로명주소', jibun_col='지번주소'
    ), axis=1
)

# df_2024 시도명 처리
df_2024['시도'] = df_2024['시도'].apply(normalize_sido)

df_2024['시장ID'] = df_2024['시도'] + ' ' + df_2024['시군구'] + ' ' + df_2024['시장명']

In [23]:
# 1. df_st_2024
print("🔎 df_st_2024 중복된 시장ID 수:", df_st_updated['시장ID'].duplicated().sum())
# @@@

# 2. df_2024
print("🔎 df_2024 중복된 시장ID 수:", df_2024['시장ID'].duplicated().sum())

🔎 df_st_2024 중복된 시장ID 수: 13
🔎 df_2024 중복된 시장ID 수: 0


In [50]:
df_st_updated[df_st_updated['시장명'] == '홍천중앙시장'][['시장명', '소재지도로명주소', '시장ID']]

,시장명,소재지도로명주소,시장ID
54,홍천중앙시장,강원도 홍천군 홍천읍 신장대리 5-3,강원도 홍천군 홍천중앙시장
55,홍천중앙시장,강원도 홍천군 홍천읍 홍천로8길 13,강원도 홍천군 홍천중앙시장


In [24]:
df_st_updated[df_st_updated['시장ID'].duplicated()]

,시장명,시장유형,소재지도로명주소,시장개설주기,소재지지번주소,점포수,사용가능상품권,공중화장실 보유여부,주차장 보유여부,개설년도,데이터기준일자,소재지도로명주소_정규화,시장명_정규화,시장명_원본,시장명_변경됨,시도,시군구,시장ID
55,홍천중앙시장,상설+정기,강원도 홍천군 홍천읍 홍천로8길 13,5일,강원 홍천군 홍천읍 신장대리 8-130번지,111,온누리상품권,Y,N,2012.0,2015-12-31,강원도홍천군홍천읍홍천로8길13,홍천시장,홍천시장,True,강원도,홍천군,강원도 홍천군 홍천중앙시장
102,미래타운제2종합시장,상설,경기도 성남시 분당구 불정로 386번길 10,상설,경기 성남시 서현동 307,95,NaN,Y,Y,2006.0,2015-12-31,경기도성남시분당구불정로386번길10,미래타운제1종합시장,미래타운제1종합시장,True,경기도,성남시,경기도 성남시 미래타운제2종합시장
151,중앙시장,상설,경기도 안양시 만안구 범천로 196,상설,경기 안양시 안양4동 676,480,온누리상품권,Y,N,2006.0,2015-12-31,경기도안양시만안구범천로196,안양중앙인정시장,안양중앙인정시장,True,경기도,안양시,경기도 안양시 중앙시장
431,예천중앙시장,상설+정기,경상북도 예천군 예천읍 시장로 109,5일,경북 예천군 예천읍 동본리 533,73,온누리상품권,N,N,2011.0,2015-12-31,경상북도예천군예천읍시장로109,예천중앙시장,예천중앙시장,False,경상북도,예천군,경상북도 예천군 예천중앙시장
515,광주양동시장,상설,광주광역시 서구 천변좌로 294,상설,광주 서구 양동 135-1,231,온누리상품권,Y,Y,1996.0,2015-12-31,광주광역시서구천변좌로294,양동산업용품시장,양동산업용품시장,True,광주광역시,서구,광주광역시 서구 광주양동시장
573,복현전통신시장,상설,대구광역시 북구 경진로1길 78,상설,대구 북구 복현동 462,180,NaN,Y,Y,NaN,2015-12-31,대구광역시북구경진로1길78,복현종합시장,복현종합시장,True,대구광역시,북구,대구광역시 북구 복현전통신시장
587,삼성시장,상설,대구광역시 북구 칠성시장로 5길 22,상설,대구 북구 칠성1가 81,139,온누리상품권,Y,N,2006.0,2015-12-31,대구광역시북구칠성시장로5길22,칠성원시장,칠성원시장,True,대구광역시,북구,대구광역시 북구 삼성시장
625,서문시장제1지구포목시장,상설,대구광역시 중구 큰장로 28길 28,상설,대구 중구 대신동 115-377,677,온누리상품권,Y,N,1974.0,2015-12-31,대구광역시중구큰장로28길28,서문시장제4지구,서문시장제4지구,True,대구광역시,중구,대구광역시 중구 서문시장제1지구포목시장
641,신중앙시장,상설,대전광역시 동구 중앙로 200번지,상설,대전 동구 원동 38-1,107,온누리상품권,Y,N,1964.0,2015-12-31,대전광역시동구중앙로200번지,중앙도매시장,중앙도매시장,True,대전광역시,동구,대전광역시 동구 신중앙시장
855,화곡중앙시장,상설,서울특별시 강서구 월정로30길 69,상설,서울 강서구 화곡 370,11,온누리상품권,Y,N,2005.0,2015-12-31,서울특별시강서구월정로30길69,화곡중앙시장,화곡중앙시장,False,서울특별시,강서구,서울특별시 강서구 화곡중앙시장


### 병합 전 중복된 시장ID 처리
> - df_2024 주소와 중복된 시장ID인 주소 유사도 비교

In [25]:
from difflib import SequenceMatcher

def address_similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

# df_2024 주소 정규화
df_2024_addr_map = df_2024[['시장ID', '도로명주소']].drop_duplicates()
df_2024_addr_map['도로명주소_정규화'] = df_2024_addr_map['도로명주소'].apply(normalize_address)
addr_map = df_2024_addr_map.set_index('시장ID')['도로명주소_정규화'].to_dict()

# df_st_updated 주소 정규화
df_st_updated['소재지도로명주소_정규화'] = df_st_updated['소재지도로명주소'].apply(normalize_address)

# 중복 시장ID 목록
dup_ids = df_st_updated[df_st_updated['시장ID'].duplicated(keep=False)]['시장ID'].unique()

# 정제된 결과 저장용 리스트
filtered_rows = []

for sid in dup_ids:
    group = df_st_updated[df_st_updated['시장ID'] == sid].copy()
    if sid in addr_map:
        base_addr = addr_map[sid]
        group['addr_sim'] = group['소재지도로명주소_정규화'].apply(lambda x: address_similar(x, base_addr))
        best_row = group.sort_values('addr_sim', ascending=False).iloc[0]
        filtered_rows.append(best_row.to_dict())
    else:
        # 기준 주소 없음 → 모두 유지
        filtered_rows.extend(group.to_dict('records'))

# DataFrame으로 변환
filtered_df = pd.DataFrame(filtered_rows)

# 나머지 중복 없는 ID 행과 합치기
non_dup_df = df_st_updated[~df_st_updated['시장ID'].isin(dup_ids)]
df_st_final = pd.concat([non_dup_df, filtered_df], ignore_index=True)

# 확인
print("✅ 중복 제거 후 최종 행 수:", len(df_st_final))
print("남은 중복 시장ID 수:", df_st_final['시장ID'].duplicated().sum())

✅ 중복 제거 후 최종 행 수: 1427
남은 중복 시장ID 수: 1


| 항목                       | 값         |
| ------------------------ | --------- |
| 원래 행 수 (`df_st_updated`) | 1439행     |
| 중복 제거 후 행 수              | **1427행** |
| 제거된 중복 행 수               | **12행**   |
| 남아 있는 중복 시장ID 수          | **1개**    |

### 유일하게 정말 중복된 시장ID는 drop

In [26]:
df_st_final[df_st_final['시장ID'].duplicated(keep=False)]

,시장명,시장유형,소재지도로명주소,시장개설주기,소재지지번주소,점포수,사용가능상품권,공중화장실 보유여부,주차장 보유여부,개설년도,데이터기준일자,소재지도로명주소_정규화,시장명_정규화,시장명_원본,시장명_변경됨,시도,시군구,시장ID,addr_sim
1424,성대시장,상설,서울특별시 동작구 상도로 102,상설,서울 동작구 상도3동 324-1,68,온누리상품권,Y,Y,NaN,2015-12-31,서울특별시동작구상도로102,성대시장,성대시장,False,서울특별시,동작구,서울특별시 동작구 성대시장,NaN
1425,성대시장,상설,서울특별시 동작구 상도로 102,상설,서울 동작구 상도동 324-1,88,온누리상품권,N,N,2015.0,2015-12-31,서울특별시동작구상도로102,성대시장,성대전통시장,True,서울특별시,동작구,서울특별시 동작구 성대시장,NaN


In [27]:
df_st_final = df_st_final.drop_duplicates(subset='시장ID') # 상도시장 그냥 하나만 남기기

### 필요한 컬럼 선택
> - 동일한 컬럼이라도 시장유형은 df_st가 더 자세하여 선택

In [28]:
# 필요한 컬럼만 선택하여 병합
cols_st = ['시장ID', '시장유형', '시장개설주기',
           '점포수', '공중화장실 보유여부', '사용가능상품권']
cols_24 = ['시장ID', '시장명', '도로명주소', '시도', '시군구',
           '아케이드 보유 여부', '엘리베이터_에스컬레이터_보유여부', '고객지원센터 보유 여부',
           '시장전용 고객주차장_보유여부', '종합콜센터_보유여부',
           '스프링쿨러 보유 여부', '화재감지기 보유여부', '자동심장충격기_보유여부',
           '유아놀이방_보유여부', '수유센터_보유여부',
           '물품보관함_보유여부', '자전거보관함_보유여부',
           '외국인 안내센터_보유여부']

In [29]:
df_st_final['시장유형'].unique()

array(['상설', '상설+정기', '정기'], dtype=object)

In [30]:
df_2024['시장 유형'].unique()

array(['전통시장'], dtype=object)

In [31]:
df_st_final.columns

Index(['시장명', '시장유형', '소재지도로명주소', '시장개설주기', '소재지지번주소', '점포수', '사용가능상품권',
       '공중화장실 보유여부', '주차장 보유여부', '개설년도', '데이터기준일자', '소재지도로명주소_정규화', '시장명_정규화',
       '시장명_원본', '시장명_변경됨', '시도', '시군구', '시장ID', 'addr_sim'],
      dtype='object')

In [32]:
df_2024.columns

Index(['시장코드', '시장명', '시장 유형', '지번주소', '도로명주소', '시도', '시군구', '아케이드 보유 여부',
       '엘리베이터_에스컬레이터_보유여부', '고객지원센터 보유 여부', '스프링쿨러 보유 여부', '화재감지기 보유여부',
       '유아놀이방_보유여부', '종합콜센터_보유여부', '고객휴게실_보유여부', '수유센터_보유여부', '물품보관함_보유여부',
       '자전거보관함_보유여부', '체육시설_보유여부', '간이 도서관_보유여부', '쇼핑카트_보유여부', '외국인 안내센터_보유여부',
       '고객동선통로_보유여부', '방송센터_보유여부', '문화교실_보유여부', '공동물류창고_보유여부',
       '시장전용 고객주차장_보유여부', '교육장_보유여부', '회의실_보유여부', '자동심장충격기_보유여부', '시장ID'],
      dtype='object')

In [33]:
df_2024['시장 유형'].unique()

array(['전통시장'], dtype=object)

### 시장ID 통일

In [34]:
unmatched_ids = set(df_2024['시장ID']) - set(df_st_final['시장ID'])
print("⛔ df_st_2024에 없는 시장ID 수:", len(unmatched_ids))
print("예시:", list(unmatched_ids))

unmatched_ids = set(df_st_final['시장ID']) - set(df_2024['시장ID'])
print("⛔ df_2024에 없는 시장ID 수:", len(unmatched_ids))
print("예시:", list(unmatched_ids))

⛔ df_st_2024에 없는 시장ID 수: 286
예시: ['세종특별자치시 금남면 세종대평시장', '경기도 안산시 안산패션타운전통시장', '대구광역시 수성구 지산목련시장(지산종합시장)', '충청남도 태안군 태안서부시장', '충청북도 청주시 청주사창시장', '울산광역시 북구 화봉시장(화봉종합시장)', '경상남도 창원시 마산수산시장', '경상남도 창원시 합성대로상가', '강원도 홍천군 홍천시장', '부산광역시 동구 범일골목시장', '울산광역시 동구 남목마성시장', '서울특별시 광진구 신성종합시장(건물형)', '경기도 안산시 신안프라자전통시장', '경상남도 진주시 새서부시장', '경상남도 창원시 덕산그린피아상가', '충청북도 충주시 성서중심시장(성서시장,문화의거리 통합)', '경상북도 경주시 건천시장', '대구광역시 달성군 현풍백년도깨비시장', '경기도 평택시 송탄시장', '경상남도 창원시 중부시장', '서울특별시 중랑구 장미제일시장', '제주특별자치도 제주시 한림민속오일시장(한림오일시장)', '강원도 평창군 평창올림픽시장(평창전통시장)', '경상북도 경주시 황남상가시장', '서울특별시 동대문구 청량리농수산물시장', '강원도 강릉시 주문진좌판풍물시장', '경상북도 문경시 점촌전통시장 (구 신흥시장)', '경상북도 김천시 부곡시장', '대전광역시 중구 산성뿌리전통시장', '전라남도 여수시 중앙선어시장', '서울특별시 도봉구 쌍문시장', '제주특별자치도 제주시 도남시장', '경기도 화성시 궁평항해오름수산시장', '경상남도 창원시 지귀시장', '제주특별자치도 서귀포시 서귀포향토오일시장', '부산광역시 사상구 사상시장', '부산광역시 동구 부산남문시장', '경기도 성남시 미래그린시장', '전라북도 정읍시 정읍연지시장', '경기도 성남시 모란전통기름시장', '경상북도 예천군 예천상설시장', '경기도 성남시 모란종합시장', '서울특별시 금천구 대명여울빛거리시장', '전라북도 전주시 서부시장상점가(서부시장 통합)', '대구광역시 북구 전기재료관', '경상북도 청송군 안덕시장'

In [35]:
# 병합: df_2024 + df_st_2024
merged = pd.merge(
    df_2024[cols_24],
    df_st_final[cols_st],
    on='시장ID',
    how='left'
)

print(merged.columns)

Index(['시장ID', '시장명', '도로명주소', '시도', '시군구', '아케이드 보유 여부', '엘리베이터_에스컬레이터_보유여부',
       '고객지원센터 보유 여부', '시장전용 고객주차장_보유여부', '종합콜센터_보유여부', '스프링쿨러 보유 여부',
       '화재감지기 보유여부', '자동심장충격기_보유여부', '유아놀이방_보유여부', '수유센터_보유여부', '물품보관함_보유여부',
       '자전거보관함_보유여부', '외국인 안내센터_보유여부', '시장유형', '시장개설주기', '점포수', '공중화장실 보유여부',
       '사용가능상품권'],
      dtype='object')


In [36]:
merged.shape

(1388, 23)

### 중복된 시장명 없음 확인

In [37]:
# 시장명별 행 수 확인
duplicated_counts = merged['시장ID'].value_counts()
duplicated = duplicated_counts[duplicated_counts > 1]

print("중복된 시장명 수:", len(duplicated))
print("예시:")
print(duplicated.head())

중복된 시장명 수: 0
예시:
Series([], Name: count, dtype: int64)


# NaN 확인하기

In [38]:
print("NaN 개수:\n", merged.isnull().sum().sort_values(ascending=False))

NaN 개수:
 사용가능상품권              470
점포수                  286
시장유형                 286
시장개설주기               286
공중화장실 보유여부           286
시장명                    0
시장ID                   0
엘리베이터_에스컬레이터_보유여부      0
아케이드 보유 여부             0
시군구                    0
시도                     0
도로명주소                  0
고객지원센터 보유 여부           0
종합콜센터_보유여부             0
시장전용 고객주차장_보유여부        0
수유센터_보유여부              0
유아놀이방_보유여부             0
자동심장충격기_보유여부           0
화재감지기 보유여부             0
스프링쿨러 보유 여부            0
외국인 안내센터_보유여부          0
물품보관함_보유여부             0
자전거보관함_보유여부            0
dtype: int64


In [39]:
merged['시장ID'].nunique() # 시장 ID를 고유한 키로 설정

1388

# 저장

In [40]:
merged.to_csv(
    "/content/drive/MyDrive/1_public/markets_data/markets_cleaned.csv",
    index=False,
    encoding='utf-8-sig'
)

# 프로세스

In [41]:
# # 크롤링 함수 작성
# def get_review_data(market_name):
#     url = f"https://map.naver.com/v5/search/{market_name}"
#     response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
#     soup = BeautifulSoup(response.text, 'html.parser')
#     rating, reviews = None, None
#     try:
#         rating = soup.select_one('.rating_class').text
#         reviews = soup.select_one('.review_count_class').text
#     except:
#         pass
#     return rating, reviews

In [42]:
# # 리뷰 데이터 병합
# market_df[['평점', '리뷰수']] = market_df['시장명'].apply(lambda x: pd.Series(get_review_data(x)))

In [43]:
# # 저장
# conn = sqlite3.connect('market_review.db')
# market_df.to_sql('market_info', conn, if_exists='replace', index=False)
# conn.close()

---

### 인코딩 감지

In [44]:
# !pip install chardet

In [45]:
# import chardet

# with open('/content/drive/MyDrive/1_public/markets_data/markets_st_2024.csv', 'rb') as f:
#     rawdata = f.read(10000)
#     result = chardet.detect(rawdata)
#     print(result)

---